<a href="https://colab.research.google.com/github/CharlesPoletowin/YCBS-273/blob/master/FashionMinist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is the demo of fashion minist dataset

## import library I need

In [0]:
%matplotlib inline
from pathlib import Path
import requests
import gzip
import numpy as np
import pandas as pd
import math
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

## download and prepare for the dataset

In [0]:
DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

def get_data(url, file_name, offset):
  content = requests.get(url).content
  (PATH / file_name).open("wb").write(content)
  with gzip.open((PATH / file_name).as_posix(), "rb") as f:
    data = np.frombuffer(f.read(), dtype=np.uint8, offset=offset)
  return data

y_train = get_data("http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz", "train-labels", 8).astype('int64')
x_train = get_data("http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz", "train-images", 16).reshape(len(y_train), 784).astype('float32')
y_test = get_data("http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz", "test-labels", 8).astype('int64')
x_test = get_data("http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz", "test-images", 16).reshape(len(y_test), 784).astype('float32')
x_train, y_train, x_test, y_test = map(
    torch.tensor, (x_train, y_train, x_test, y_test)
)
n = x_train.shape[0]
x_test=x_test.cuda()
x_train=x_train.cuda()
y_test=y_test.cuda()
y_train=y_train.cuda()

http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz

http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz

http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz

http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz

## the initial model of cnn

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [0]:
cnn = CNN()
cnn.cuda()

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

In [0]:
def train_model(model, x_train, y_train, x_test, y_test, epochs=250, batch_size=256, lr=0.01, weight_decay=0):
  # data
  train_dataset = TensorDataset(x_train, y_train)
  train_data_loader = DataLoader(train_dataset, batch_size=batch_size)
  
  # loss function
  loss_func = F.cross_entropy

  # optimizer
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

  # training loop
  for epoch in range(epochs):
    for xb, yb in train_data_loader:
      
      pred = model(xb)
      loss = loss_func(pred, yb)

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

    y_pred = model(x_test)
    acc = accuracy_score(torch.argmax(y_pred, dim=1).cpu().detach().numpy(), y_test.cpu())
    
    print('Loss at epoch %d : %f, test acc: %f' % (epoch, loss, acc))

## train the model

In [0]:
train_model(cnn, x_train.view(-1,1,28,28), y_train, x_test.view(-1,1,28,28), y_test,epochs=30,lr=0.001)

Loss at epoch 0 : 0.296137, test acc: 0.864400
Loss at epoch 1 : 0.217342, test acc: 0.882700
Loss at epoch 2 : 0.187078, test acc: 0.886800
Loss at epoch 3 : 0.172404, test acc: 0.889700
Loss at epoch 4 : 0.159349, test acc: 0.888700
Loss at epoch 5 : 0.144943, test acc: 0.886800
Loss at epoch 6 : 0.128845, test acc: 0.886200
Loss at epoch 7 : 0.107657, test acc: 0.886100
Loss at epoch 8 : 0.096159, test acc: 0.886400
Loss at epoch 9 : 0.085074, test acc: 0.886200
Loss at epoch 10 : 0.075651, test acc: 0.887000
Loss at epoch 11 : 0.062632, test acc: 0.887400
Loss at epoch 12 : 0.056205, test acc: 0.886100
Loss at epoch 13 : 0.049004, test acc: 0.889200
Loss at epoch 14 : 0.045795, test acc: 0.889600
Loss at epoch 15 : 0.038661, test acc: 0.889700
Loss at epoch 16 : 0.037889, test acc: 0.891100
Loss at epoch 17 : 0.034072, test acc: 0.891900
Loss at epoch 18 : 0.028304, test acc: 0.895500
Loss at epoch 19 : 0.025917, test acc: 0.899300
Loss at epoch 20 : 0.026632, test acc: 0.899400
Lo

In [0]:
train_model(cnn, x_train.view(-1,1,28,28), y_train, x_test.view(-1,1,28,28), y_test,epochs=30,lr=0.0001)

Loss at epoch 0 : 0.011768, test acc: 0.909400
Loss at epoch 1 : 0.011026, test acc: 0.909800
Loss at epoch 2 : 0.010690, test acc: 0.909500
Loss at epoch 3 : 0.010585, test acc: 0.909300
Loss at epoch 4 : 0.010420, test acc: 0.908800
Loss at epoch 5 : 0.010312, test acc: 0.908900
Loss at epoch 6 : 0.010241, test acc: 0.908600
Loss at epoch 7 : 0.010193, test acc: 0.908000
Loss at epoch 8 : 0.010093, test acc: 0.908300
Loss at epoch 9 : 0.010107, test acc: 0.908200
Loss at epoch 10 : 0.010012, test acc: 0.908400
Loss at epoch 11 : 0.009913, test acc: 0.908100
Loss at epoch 12 : 0.009814, test acc: 0.908100
Loss at epoch 13 : 0.009638, test acc: 0.908400
Loss at epoch 14 : 0.009567, test acc: 0.908300
Loss at epoch 15 : 0.009448, test acc: 0.908300
Loss at epoch 16 : 0.009296, test acc: 0.908500
Loss at epoch 17 : 0.009217, test acc: 0.908700
Loss at epoch 18 : 0.008958, test acc: 0.908400
Loss at epoch 19 : 0.008811, test acc: 0.908300
Loss at epoch 20 : 0.008685, test acc: 0.908400
Lo

## another model 

In [0]:
class FashionCNN(nn.Module):
    
    def __init__(self):
        super(FashionCNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

In [0]:
newmodel=FashionCNN()
newmodel.cuda()

FashionCNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)

In [0]:
train_model(newmodel, x_train.view(-1,1,28,28), y_train, x_test.view(-1,1,28,28), y_test,epochs=30,lr=0.001)

Loss at epoch 0 : 0.255372, test acc: 0.849100
Loss at epoch 1 : 0.204697, test acc: 0.879400
Loss at epoch 2 : 0.227691, test acc: 0.891400
Loss at epoch 3 : 0.200622, test acc: 0.887000
Loss at epoch 4 : 0.196673, test acc: 0.895000
Loss at epoch 5 : 0.180228, test acc: 0.893900
Loss at epoch 6 : 0.149391, test acc: 0.894500
Loss at epoch 7 : 0.135818, test acc: 0.898800
Loss at epoch 8 : 0.135960, test acc: 0.898100
Loss at epoch 9 : 0.103918, test acc: 0.895900
Loss at epoch 10 : 0.095497, test acc: 0.904800
Loss at epoch 11 : 0.067835, test acc: 0.906300
Loss at epoch 12 : 0.098062, test acc: 0.893000
Loss at epoch 13 : 0.037792, test acc: 0.895600
Loss at epoch 14 : 0.053417, test acc: 0.900900
Loss at epoch 15 : 0.054488, test acc: 0.896000
Loss at epoch 16 : 0.032922, test acc: 0.903700
Loss at epoch 17 : 0.032892, test acc: 0.904200
Loss at epoch 18 : 0.048263, test acc: 0.903900
Loss at epoch 19 : 0.038989, test acc: 0.908600
Loss at epoch 20 : 0.022890, test acc: 0.909400
Lo

In [0]:
train_model(newmodel, x_train.view(-1,1,28,28), y_train, x_test.view(-1,1,28,28), y_test,epochs=300,lr=0.0001)

Loss at epoch 0 : 0.005355, test acc: 0.907900
Loss at epoch 1 : 0.008211, test acc: 0.908600
Loss at epoch 2 : 0.007290, test acc: 0.909600
Loss at epoch 3 : 0.004375, test acc: 0.908300
Loss at epoch 4 : 0.003830, test acc: 0.908700
Loss at epoch 5 : 0.003673, test acc: 0.908900
Loss at epoch 6 : 0.003412, test acc: 0.909200
Loss at epoch 7 : 0.002617, test acc: 0.908400
Loss at epoch 8 : 0.003572, test acc: 0.907800
Loss at epoch 9 : 0.001832, test acc: 0.907600
Loss at epoch 10 : 0.001966, test acc: 0.907800
Loss at epoch 11 : 0.001723, test acc: 0.908500
Loss at epoch 12 : 0.001343, test acc: 0.905500
Loss at epoch 13 : 0.000842, test acc: 0.907000
Loss at epoch 14 : 0.001365, test acc: 0.908400
Loss at epoch 15 : 0.001329, test acc: 0.906800
Loss at epoch 16 : 0.000672, test acc: 0.906300
Loss at epoch 17 : 0.000666, test acc: 0.907500
Loss at epoch 18 : 0.000310, test acc: 0.906500
Loss at epoch 19 : 0.000225, test acc: 0.906100
Loss at epoch 20 : 0.002236, test acc: 0.905300
Lo